<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #7: Transfer Object Detection models to Zenodo</h1>
<h3 align="right">Written by KSO Team</h3>

# Set up KSO requirements

### Install kso-object-detection and its requirements

In [ ]:
from IPython.display import clear_output
import os
import sys

try:
    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    IN_COLAB = True
    print("Running in Colab...")

    # Clone repo
    !git clone --recurse-submodules -b master https://github.com/ocean-data-factory-sweden/kso.git
    %pip install -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/yolov5_tracker/yolov5/requirements.txt) -qr <(sed '/Pillow/d;/ipywidgets/d' kso/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Navigate to the correct folder
    os.chdir("kso/tutorials")

except:
    IN_COLAB = False


# Ensure widgets are shown properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

clear_output()
if IN_COLAB == True:
    print("Running in Colab: All packages are installed and ready to go!")
else:
    print("Running locally... you're good to go!")

In [ ]:
# Replace nearest neighbours script with custom version (due to relative path issue)
import shutil

shutil.copy(
    "../src/multi_tracker_zoo.py",
    "../yolov5_tracker/trackers/strong_sort/multi_tracker_zoo.py",
)

### Import specific requirements

In [ ]:
try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")

        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
import kso_utils.zenodo_utils as zenodo_utils
from kso_utils.project import ProjectProcessor, MLProjectProcessor

clear_output()
print("Packages loaded successfully")

# Find model and add to Zenodo

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

In [ ]:
project = p_utils.find_project(project_name=project_name.value)

In [ ]:
# Initialise pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

### Choose the model

In [ ]:
model = mlp.choose_model()

### Choose folder to download the model

In [ ]:
download_dir = kso_widgets.choose_folder(".", "downloaded model")

### Download model

In [ ]:
artifact_dir = mlp.get_model(model.value, download_dir.selected)

# ZENODO Upload

### Provide access token to your account

In [ ]:
# It is found in API settings by going to your Zenodo profile and clicking on Applications.
ACCESS_TOKEN = ""

### Find archive for upload to Zenodo

In [ ]:
archive_dir = kso_widgets.choose_folder(".", "archive for upload")

### Add archive to upload

In [ ]:
depo_id = zenodo_utils.upload_archive(ACCESS_TOKEN, artifact_dir=artifact_dir)

### Indicate database title

In [ ]:
upload_title = t_utils.choose_text("title")

### Write database description

In [ ]:
upload_description = t_utils.choose_text("description")

### Indicate database authors together with  their organisations

In [ ]:
authors = t_utils.WidgetMaker()
authors

In [ ]:
authors.author_dict

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Add dataset metadata to upload</font> { vertical-output: true }
zenodo_utils.add_metadata_zenodo_upload(
    ACCESS_TOKEN,
    depo_id,
    upload_title.value,
    upload_description.value,
    authors.author_dict,
)

In [ ]:
# END